# Sentinel Hub Statistical API
    
The Statistical API enables you to get statistics calculated based on satellite imagery without having to download images. In your Statistical API request, you can specify your area of interest, time period, evalscript and which statistical measures should be calculated. The requested statistics are returned in the API response. Using Statistical API you can calculate the percentage of cloudy pixels for a given area of interest and time period, or calculate mean, standard deviation, and histogram of band values for a parcel in a given time period.<br>



**Note: Cloud mask values have been included along with the obtained statistics values of Sentinel-2 indices in the following examples.**


In [17]:
# import libraries
%matplotlib inline

import json
import datetime as dt
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import geopandas as gpd

from sentinelhub import (
    SentinelHubStatistical,
    DataCollection,
    CRS,
    BBox,
    bbox_to_dimensions,
    Geometry,
    SHConfig,
    parse_time,
    parse_time_interval,
    SentinelHubStatisticalDownloadClient,
)

# sentinel hub configurations
For the instance ID

In [29]:
from sentinelhub import SHConfig

config = SHConfig()

config.instance_id = '3704e2d2-43dc-4f34-a563-13ec19dff111'
config.sh_client_id = '65eef2f2-65ef-4e9b-9004-e54e51622279'
config.sh_client_secret = 'RxAwy:lwV.mi,]Ae}:GzFTQ/:]&v@|ra-Zx{O@c['

from sentinelhub import SHConfig


INSTANCE_ID = ''  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

Helper Function

def stats_to_df(stats_data):
    """Transform Statistical API response into a pandas.DataFrame"""
    df_data = []

    for single_data in stats_data["data"]:
        df_entry = {}
        is_valid_entry = True

        df_entry["interval_from"] = parse_time(single_data["interval"]["from"]).date()
        df_entry["interval_to"] = parse_time(single_data["interval"]["to"]).date()

        for output_name, output_data in single_data["outputs"].items():
            for band_name, band_values in output_data["bands"].items():

                band_stats = band_values["stats"]
                if band_stats["sampleCount"] == band_stats["noDataCount"]:
                    is_valid_entry = False
                    break

                for stat_name, value in band_stats.items():
                    col_name = f"{output_name}_{band_name}_{stat_name}"
                    if stat_name == "percentiles":
                        for perc, perc_val in value.items():
                            perc_col_name = f"{col_name}_{perc}"
                            df_entry[perc_col_name] = perc_val
                    else:
                        df_entry[col_name] = value

        if is_valid_entry:
            df_data.append(df_entry)

    return pd.DataFrame(df_data)

## Parameters of FIS request


* `layer` -   name of the layer defined in Sentinel Hub Configurator.
* `geometry_list` - list of geometry objects (BBox or Geometry), statistics will be calculated for each of them separately
* `time` - statistics will be calculated for each acquisition in the give time interval separately 
* `resolution` - spatial resolution on which to calculate statistics 
* `data_folder` - optional parameter for specifying location where the data should be saved locally 
* `bins` - The number of bins (a positive integer) in the histogram. When this parameter is absent, no histogram is computed.
* `histogram_type` - ways of dividing values into bins currently supported: EQUIDISTANT, EQUALFREQUENCY, or STREAMING

## Sentinel Hub Configuration

For the Sentinel-5P layer NO2, refer to **FIS-request** configuration instance

In [30]:
# read single_region geojson in geopandas
single_region = gpd.read_file("CALABARZON (Region IV-A).geojson")
single_region_geometry = Geometry(single_region.geometry.values[0], crs=CRS.WGS84)
single_region

,geometry
0,"POLYGON ((121.70654 15.20990, 121.12152 14.511..."


In [31]:
yearly_time_interval = "2020-06-20", "2022-05-20"

no2_evalscript = """
 //VERSION=3
var minVal = 0.0;
var maxVal = 0.0001;
var diff = maxVal - minVal;
const map = [
	[minVal, 0x00007f], 
	[minVal + 0.125 * diff, 0x0000ff],
	[minVal + 0.375 * diff, 0x00ffff],
	[minVal + 0.625 * diff, 0xffff00],
	[minVal + 0.875 * diff, 0xff0000],
	[maxVal, 0x7f0000]
]; 

const visualizer = new ColorRampVisualizer(map)
function setup() {
   return {
    input: ["NO2","dataMask"],
    output: { bands: 4 }
  };
}

function evaluatePixel(samples) {
   const [r, g, b] = visualizer.process(samples.NO2);
   return [r, g, b, samples.dataMask];
}"""

aggregation = SentinelHubStatistical.aggregation(
    evalscript=no2_evalscript, time_interval=yearly_time_interval, aggregation_interval="P1D", resolution=(10, 10)
)

input_data = SentinelHubStatistical.input_data(DataCollection.SENTINEL5P)

histogram_calculations = {"no2": {"histograms": {"default": {"nBins": 20, "lowEdge": -1.0, "highEdge": 1.0}}}}

no2_requests = []

for geo_shape in single_region.geometry.values:
    request = SentinelHubStatistical(
        aggregation=aggregation,
        input_data=[input_data],
        geometry=Geometry(geo_shape, crs=CRS(single_region_geometry.crs)),
        calculations=histogram_calculations,
        config=config,
    )
    no2_requests.append(request)

# initialize request to obtain basic stats and histogram values
# warning if the timeline is to long there if error timeout at their server
single_region_geometry_n = FisRequest(
    data_collection=DataCollection.SENTINEL5P,
    layer='NO2',
    geometry_list=[single_region_geometry],
    time=('2020-06-20', '2022-05-20'),
    resolution='1000m',
    config=config
)

single_region_geometry_stats_n = single_region_geometry_n.get_data()

In [32]:
%%time

download_requests = [no2_request.download_list[0] for no2_request in no2_requests]

client = SentinelHubStatisticalDownloadClient(config=config)

no2_stats = client.download(download_requests)

len(no2_stats)

DownloadFailedException: Failed to download from:
https://creodias.sentinel-hub.com/api/v1/statistics
with HTTPError:
503 Server Error: Service Unavailable for url: https://creodias.sentinel-hub.com/api/v1/statistics
Server response: ""

In [ ]:
no2_dfs = [stats_to_df(polygon_stats) for polygon_stats in no2_stats]

for df, land_type in zip(no2_dfs, polygons_gdf["land_type"].values):
    df["land_type"] = land_type

no2_df = pd.concat(no2_dfs)

ndvi_df

In [ ]:
# request will return json output
single_region_geometry_stats_n

In [ ]:
# Data Frame Cleaning
single_region_stats_clean = clean_df(single_region_geometry_stats_n)
single_region_stats_clean

In [ ]:
# Export to CSV
single_region_stats_clean.to_csv('regional_no2_results/ARMM_region_stats_clean.csv', index=False)

## END